In [1]:
import ee
import geemap
import pandas as pd
import dtale
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedKFold,train_test_split
from sklearn.preprocessing import MinMaxScaler,PowerTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error,explained_variance_score,mean_absolute_percentage_error
import glob
import dask.dataframe as dd
from tpot import TPOTRegressor
import xarray as xr
from scipy import stats
from sklearn.svm import SVR
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression,ElasticNetCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import plot_partial_dependence
from warnings import filterwarnings
filterwarnings('ignore')
import plotly
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import time
import plotly.express as px
import chart_studio.plotly as py
import folium
from folium.plugins import HeatMap
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
init_notebook_mode(connected = True)
from folium.plugins import FastMarkerCluster
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import os
from sklearn.inspection import permutation_importance

In [2]:
Map_S5P = geemap.Map()
Map_S5P

Enter verification code: 4/1AX4XfWgUpqcVRWloCi4zMb4rsbFIKHgw86bUB9OokjGqOAM4NcLxLlFjhiA

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [24]:
df = geemap.csv_to_pandas('argo_one_float_surface.csv')
df

,PLATFORM_CODE,DATE (YYYY-MM-DDTHH:MI:SSZ),DATE_QC,LATITUDE,LONGITUDE,POSITION_QC,PRES (decibar),PRES_QC,PSAL (psu),PSAL_QC,TEMP (degree_Celsius),TEMP_QC
0,4903353,2021-05-15T23:11:02Z,1,26.82214,-87.42920,1,0.44,1,36.235,1,27.984,1
1,4903353,2021-05-16T10:02:48Z,1,26.87335,-87.46692,1,1.04,1,36.287,1,27.651,1
2,4903353,2021-05-21T20:17:01Z,1,26.30145,-88.12453,1,1.04,1,36.454,1,26.716,1
3,4903353,2021-05-26T19:30:12Z,1,26.64623,-87.60446,1,0.48,1,36.475,1,27.504,1
4,4903353,2021-05-31T18:54:50Z,1,27.00173,-87.14219,1,1.04,1,36.335,1,27.875,1
5,4903353,2021-06-05T18:07:50Z,1,26.90511,-86.86409,1,0.96,1,36.224,1,28.571,1
6,4903353,2021-06-10T17:06:31Z,1,26.95002,-86.84199,1,1.00,1,36.301,1,29.193,1
7,4903353,2021-06-15T16:28:20Z,1,26.90381,-86.54071,1,1.04,1,36.274,1,29.209,1
8,4903353,2021-06-20T15:23:43Z,1,26.63506,-86.35426,1,1.00,1,36.164,1,28.408,1
9,4903353,2021-06-25T14:33:00Z,1,26.54115,-86.28348,1,1.00,1,36.243,1,28.780,1


In [27]:
Map_S5P.add_xy_data('argo_one_float_surface.csv', x="LONGITUDE", y="LATITUDE", layer_name="csv to ee", to_ee=True)

In [3]:
# Create a common region of interest (Los Angeles)
roi = ee.Geometry.Polygon([[[-104.3845703125, 47.07129893759146],
          [-104.3845703125, 6.011178003852488],
          [-0.3220703125000135, 6.011178003852488],
          [-0.3220703125000135, 47.07129893759146]]])

In [15]:
# Hycom Data COllection
image_hycom = ee.Image('HYCOM/sea_water_velocity/2021040700').divide(1000)\
    .clip(roi)

Map_S5P.addLayer(image_hycom.select('velocity_u_0').hypot(image_hycom.select('velocity_v_0')))


# This automatically pans the map to the middle of our area of interest
Map_S5P.centerObject(roi)

In [23]:
image_aqua_modis = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI')\
                  .filterDate('2016-01-01', '2016-01-31')\

    
remoteSensingReflectance = image_aqua_modis.select(['Rrs_645', 'Rrs_555', 'Rrs_443']);
remoteSensingReflectanceVis = {
  min: 0.0,
  max: 0.011,
};

Map_S5P.addLayer(
    remoteSensingReflectance, remoteSensingReflectanceVis,
    'Remote Sensing Reflectance');

# This automatically pans the map to the middle of our area of interest
Map_S5P.centerObject(roi)

In [ ]:
#traing data for July
ground_truth= image_LANDSAT8.addBands(image_S5P).addBands(image_S2_SR)

work_dir = os.path.expanduser('/Users/mdnhuda/Desktop/PARC IEM Data/Correlation for Month with Scatter')



proj = ee.Projection('EPSG:4326')
reproj_image = ground_truth.resample('bilinear').reproject(crs=proj, scale=30)


out_csv = os.path.join(work_dir, 'Combined_ESRI_S2_LandSat8_S5_Los_Angeles_TPOT July.csv')
geemap.extract_values_to_points(in_fc, reproj_image, out_csv)

df=pd.read_csv('Combined_ESRI_S2_LandSat8_S5_Los_Angeles_TPOT July.csv')

df=df.drop('system:index', axis=1)


# use pd.concat to join the new columns with your original dataframe
df = pd.concat([df,pd.get_dummies(df['Los Angele'], prefix='Class')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
df.drop(['Los Angele'],axis=1, inplace=True)


In [ ]:
dtale.show(df)

In [ ]:
#traing data for September
ground_truth_sep= image_LANDSAT8_sep.addBands(image_S5P_sep).addBands(image_S2_SR_sep)

work_dir = os.path.expanduser('/Users/mdnhuda/Desktop/PARC IEM Data/Correlation for Month with Scatter')
in_shp = os.path.join(work_dir, 'LA_Shp.shp')

in_fc = geemap.shp_to_ee(in_shp)
Map_S5P_sep.addLayer(in_fc, {}, 'Shape FIle Geometry')


proj = ee.Projection('EPSG:4326')
reproj_image_sep = ground_truth_sep.resample('bilinear').reproject(crs=proj, scale=30)


out_csv = os.path.join(work_dir, 'Combined_ESRI_S2_LandSat8_S5_Los_Angeles_TPOT Sep.csv')
geemap.extract_values_to_points(in_fc, reproj_image_sep, out_csv)

df_sep=pd.read_csv('Combined_ESRI_S2_LandSat8_S5_Los_Angeles_TPOT Sep.csv')
df_sep=df_sep.drop('system:index', axis=1)

# use pd.concat to join the new columns with your original dataframe
df_sep = pd.concat([df_sep,pd.get_dummies(df_sep['Los Angele'], prefix='Class')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
df_sep.drop(['Los Angele'],axis=1, inplace=True)


In [ ]:
dtale.show(df_sep)

In [ ]:
#df_sep.to_csv ('Combined_S2_LandSat8_S5_Los_Angeles_872_September.csv', index = False, header=True)

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

corrMatrix = df.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
df.corr()['NO2_column_number_density'].sort_values().plot(kind='bar')

It looks like B1, B2, B3, B4, B10, B11, and sr-aerosol strongly correlate with the NO2 values. It will be good to see how do they perform in a fitted model.

In [ ]:
dtale.show(df)

In [ ]:
df.isnull().sum()

In [ ]:
X=df.drop('NO2_column_number_density',axis=1).values
y=df['NO2_column_number_density'].values

scaler=MinMaxScaler()
X=scaler.fit_transform(X)
y=scaler.fit_transform(y.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
X_sep=df_sep.drop('NO2_column_number_density',axis=1).values
y_sep=df_sep['NO2_column_number_density'].values


scaler=MinMaxScaler()
X_sep=scaler.fit_transform(X_sep)
y_sep=scaler.fit_transform(y_sep.reshape(-1, 1))

X_train_sep, X_test_sep, y_train_sep, y_test_sep = train_test_split(X_sep, y_sep, test_size=0.30, random_state=42)


In [ ]:
# cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
# model = TPOTRegressor(generations=4, population_size=50, scoring='neg_mean_absolute_error', cv=cv, verbosity=2, random_state=42, n_jobs=-1)
# model.fit(X_train, y_train)

# print('Training Accuracy : %.3f'% model.score(X_train,y_train))

# print('Test Accuracy : %.3f'%model.score(X_test,y_test))

In [ ]:
#model.export('Combined_S2_LandSat8_S5_Los_Angeles_TPOT_tpot_best_model.py')

In [ ]:
#from sklearn.ensemble import ExtraTreesRegressor
model = RandomForestRegressor(bootstrap=False, max_features=0.35000000000000003, min_samples_leaf=7, min_samples_split=20, n_estimators=100)
model.fit(X_train, y_train)
print('Training Accuracy : %.3f'% model.score(X_train,y_train))

print('Test Accuracy : %.3f'%model.score(X_test,y_test))

In [ ]:
feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.title('Feature Importance')
plt.yticks(pos, np.array(df.columns)[sorted_idx])
result = permutation_importance(model, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(result.importances[sorted_idx].T,
            vert=False, labels=np.array(df.columns)[sorted_idx])
plt.title("Permutation Importance (test set)")
fig.tight_layout()
plt.show()

In [ ]:
prediction_sep=model.predict(X_test_sep)
MSE=mean_squared_error(y_test_sep,prediction_sep)
print("Mean Squared Error =",MSE)

MAPE=mean_absolute_percentage_error(y_test_sep,prediction_sep)
print("Mean absolute percentage error =",MAPE)


fig = plt.figure(figsize=(14,6))
plt.plot(y_test_sep,marker="o", ms = 8, mfc = 'b', linestyle='None',label='Actual')
plt.plot(prediction_sep,marker="v", ms = 9,linestyle='None', mfc = 'g',label='Prediction')
plt.title('Actual vs Predicted NO2_column_number_density on Unseen Data Sep (RandomForest)')
plt.legend(loc='best',prop={'size':14})
plt.xlabel('Pixel')
plt.ylabel('NO2_column_number_density')

In [ ]:
#NO2_column_number_density=df['NO2_column_number_density']

In [ ]:
#NO2_column_number_density

In [ ]:
all_prediction=model.predict(X)

In [ ]:
fig = plt.figure(figsize=(14,6))
plt.plot(y,marker="o", ms = 8, mfc = 'b', linestyle='None',label='Actual')
plt.plot(all_prediction,marker="v", ms = 9,linestyle='None', mfc = 'g',label='Prediction')
plt.title('Actual vs Predicted NO2_column_number_density on Whole Data (RandomForest)')
plt.legend(loc='best',prop={'size':14})
plt.xlabel('Pixel')
plt.ylabel('NO2_column_number_density')

## Additional Methods follows below

In [ ]:
model_params = {
    'svm': {
        'model': SVR(gamma='auto'),
        'params' : {
            'C': [10,20,50],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestRegressor(),
        'params' : {
            'n_estimators': [5,10,15]
        }
    },
}

In [ ]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df_score = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_score

In [ ]:
#Rainforest Regression
model = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=15)
model.fit(X_train,y_train)

In [ ]:
print('Training Accuracy : %.3f'% model.score(X_train,y_train))

print('Test Accuracy : %.3f'%model.score(X_test,y_test))

In [ ]:
prediction=model.predict(X_test)
MSE=mean_squared_error(y_test,prediction)
print("Mean Squared Error =",MSE)

MAPE=mean_absolute_percentage_error(y_test,prediction)
print("Mean absolute percentage error =",MAPE)


fig = plt.figure(figsize=(14,6))
plt.plot(y_test,marker="o", ms = 8, mfc = 'b', linestyle='None',label='Actual')
plt.plot(prediction,marker="v", ms = 9,linestyle='None', mfc = 'g',label='Prediction')
plt.title('Actual vs Predicted NO2_column_number_density on Test Data (Random Forest)')
plt.legend(loc='best',prop={'size':14})
plt.xlabel('Pixel')
plt.ylabel('NO2_column_number_density')

In [ ]:
prediction_sep=model.predict(X_test_sep)
MSE=mean_squared_error(y_test_sep,prediction_sep)
print("Mean Squared Error =",MSE)

MAPE=mean_absolute_percentage_error(y_test_sep,prediction_sep)
print("Mean absolute percentage error =",MAPE)


fig = plt.figure(figsize=(14,6))
plt.plot(y_test_sep,marker="o", ms = 8, mfc = 'b', linestyle='None',label='Actual')
plt.plot(prediction_sep,marker="v", ms = 9,linestyle='None', mfc = 'g',label='Prediction')
plt.title('Actual vs Predicted NO2_column_number_density on Unseen Data Sep (Random Forest)')
plt.legend(loc='best',prop={'size':14})
plt.xlabel('Pixel')
plt.ylabel('NO2_column_number_density')

In [ ]:
NO2_column_number_density=df['NO2_column_number_density']

In [ ]:
X_all=X
all_prediction=model.predict(X_all)

In [ ]:
fig = plt.figure(figsize=(14,6))
plt.plot(y,marker="o", ms = 8, mfc = 'b', linestyle='None',label='Actual')
plt.plot(all_prediction,marker="v", ms = 9,linestyle='None', mfc = 'g',label='Prediction')
plt.title('Actual vs Predicted NO2_column_number_density on Whole Data (Random Forest)')
plt.legend(loc='best',prop={'size':14})
plt.xlabel('Pixel')
plt.ylabel('NO2_column_number_density')

In [ ]:
#X.rename(index={0: "B1", 1: "B10", 2: "B11", 3: "B2",4: "B3", 5: "B4", 6: "B5", 7: "B6", 8: "B7"})

In [ ]:
display = plot_partial_dependence(
       model, X, features=[0,1,2,3,4,5,6,7,8,9])

display.figure_.suptitle('Partial dependence of No2, with Random Forest')
display.figure_.subplots_adjust(hspace=0.5)

In [ ]:
logit_model=sm.OLS(y_train,X_train)
result=logit_model.fit()
print(result.summary())

# Linear Regression for Multioutput Regression

In [ ]:
model = LinearRegression()
# fit model
model.fit(X_train,y_train)
# make a prediction
prediction=model.predict(X_test)
# summarize prediction
#print(prediction[0,0])

##
MSE=mean_squared_error(y_test,prediction)
print("Mean Squared Error =",MSE)
MAPE=mean_absolute_percentage_error(y_test,prediction)
print("Mean absolute percentage error =",MAPE)


fig = plt.figure(figsize=(14,6))
plt.plot(y_test,marker="o", ms = 8, mfc = 'b', linestyle='None',label='Actual')
plt.plot(prediction,marker="v", ms = 9,linestyle='None', mfc = 'g',label='Prediction')
plt.title('Actual vs Predicted NO2_column_number_density on Test Data (Linear Regression)')
plt.legend(loc='best',prop={'size':14})
plt.xlabel('Pixel')
plt.ylabel('NO2_column_number_density')

fig = plt.figure(figsize=(14,6))
plt.plot(y_test,marker="o", ms = 8, mfc = 'b', linestyle='None',label='Actual')
plt.plot(prediction,marker="v", ms = 9,linestyle='None', mfc = 'g',label='Prediction')
plt.title('Actual vs Predicted tropospheric_NO2_column_number_density on Test Data (Linear Regression)')
plt.legend(loc='best',prop={'size':14})
plt.xlabel('Pixel')
plt.ylabel('Ntropospheric_NO2_column_number_density')

logit_model=sm.OLS(y_train,X_train)
result=logit_model.fit()
print(result.summary())

In [ ]:
print('Training Accuracy : %.3f'% model.score(X_train,y_train))

print('Test Accuracy : %.3f'%model.score(X_test,y_test))

# k-Nearest Neighbors for Multioutput Regression

In [ ]:
# define model
model = KNeighborsRegressor()
# fit model
model.fit(X_train,y_train)
# make a prediction
prediction=model.predict(X_test)
# summarize prediction
#print(prediction[0,0])

MSE=mean_squared_error(y_test,prediction)
print("Mean Squared Error =",MSE)
MAPE=mean_absolute_percentage_error(y_test,prediction)
print("Mean absolute percentage error =",MAPE)

fig = plt.figure(figsize=(14,6))
plt.plot(y_test,marker="o", ms = 8, mfc = 'b', linestyle='None',label='Actual')
plt.plot(prediction,marker="v", ms = 9,linestyle='None', mfc = 'g',label='Prediction')
plt.title('Actual vs Predicted NO2_column_number_density on Test Data (k-Nearest Neighbors)')
plt.legend(loc='best',prop={'size':14})
plt.xlabel('Pixel')
plt.ylabel('NO2_column_number_density')

logit_model=sm.OLS(y_train ,X_train)
result=logit_model.fit()
print(result.summary())

In [ ]:
print('Training Accuracy : %.3f'% model.score(X_train,y_train))

print('Test Accuracy : %.3f'%model.score(X_test,y_test))

# Decision Tree for Multioutput Regression

In [ ]:
# define model
model = DecisionTreeRegressor(max_depth=8,
                           min_samples_split=5,
                           max_leaf_nodes=20)
# fit model
model.fit(X_train,y_train)
# make a prediction
prediction=model.predict(X_test)
# summarize prediction
#print(prediction)

MSE=mean_squared_error(y_test,prediction)
print("Mean Squared Error =",MSE)
MAPE=mean_absolute_percentage_error(y_test,prediction)
print("Mean absolute percentage error =",MAPE)

fig = plt.figure(figsize=(14,6))
plt.plot(y_test,marker="o", ms = 8, mfc = 'b', linestyle='None',label='Actual')
plt.plot(prediction,marker="v", ms = 9,linestyle='None', mfc = 'g',label='Prediction')
plt.title('Actual vs Predicted NO2_column_number_density on Test Data (Decision Tree)')
plt.legend(loc='best',prop={'size':14})
plt.xlabel('Pixel')
plt.ylabel('NO2_column_number_density')

logit_model=sm.OLS(y_train ,X_train)
result=logit_model.fit()
print(result.summary())

In [ ]:
print('Training Accuracy : %.3f'% model.score(X_train,y_train))

print('Test Accuracy : %.3f'%model.score(X_test,y_test))

In [ ]:
X=df.drop('NO2_column_number_density',axis=1).values
y=df['NO2_column_number_density'].values

scaler=MinMaxScaler()
X=scaler.fit_transform(X)
y=scaler.fit_transform(y.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(alpha=0.85, learning_rate=0.5, loss='lad', max_depth=15, max_features=0.9500000000000001,
                                min_samples_leaf=12, min_samples_split=7, n_estimators=100, subsample=0.6500000000000001)
# fit model
model.fit(X_train,y_train)
# make a prediction
prediction=model.predict(X_test)
# summarize prediction
#print(prediction[0,0])

MSE=mean_squared_error(y_test,prediction)
print("Mean Squared Error =",MSE)
MAPE=mean_absolute_percentage_error(y_test,prediction)
print("Mean absolute percentage error =",MAPE)

fig = plt.figure(figsize=(14,6))
plt.plot(y_test,marker="o", ms = 8, mfc = 'b', linestyle='None',label='Actual')
plt.plot(prediction,marker="v", ms = 9,linestyle='None', mfc = 'g',label='Prediction')
plt.title('Actual vs Predicted NO2_column_number_density on Test Data (Decision Tree)')
plt.legend(loc='best',prop={'size':14})
plt.xlabel('Pixel')
plt.ylabel('NO2_column_number_density')

logit_model=sm.OLS(y_train,X_train)
result=logit_model.fit()
print(result.summary())

In [ ]:
print('Training Accuracy : %.3f'% model.score(X_train,y_train))

print('Test Accuracy : %.3f'%model.score(X_test,y_test))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

In [ ]:
exported_pipeline = RandomForestRegressor(bootstrap=False, max_features=0.4, min_samples_leaf=16, min_samples_split=14, n_estimators=100)


In [ ]:
exported_pipeline.fit(X_train,y_train)
print('Training Accuracy : %.3f'% model.score(X_train,y_train))

print('Test Accuracy : %.3f'%model.score(X_test,y_test))